In [ ]:
import os
import numpy as np
from skimage import io

DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/MD589/preps/CH1/full_aligned'
volume = []
for filename in sorted(os.listdir(DIR)):
    filepath = os.path.join(DIR, filename)
    volume.append(io.imread(filepath))
volume = np.array(volume)
volume = np.swapaxes(volume, 0, 2)

# Precomputed folder path
precompute_path = os.path.join(os.path.expanduser('~'), f'image_test')

# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (452, 452, 20000)

# Voxel offset
offset = (0, 0, 0)

# Layer type
layer_type = 'image'

In [3]:
import os
import sys
import numpy as np
sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.sqlcontroller import SqlController

DIR = os.path.join(os.path.expanduser('~'))
animal = 'MD589'
downsample_factor = 32

volume_path = os.path.join(DIR, f'{animal}_annotations_down{downsample_factor}.npy')

# Numpy array whose shape should match (x, y, z, channel) or just (x, y, z) if it's single channel
with open(volume_path, 'rb') as file:
    volume = np.load(file)
print(volume.shape)

# Precomputed folder path
precompute_path = os.path.join(DIR, f'{animal}_annotations_down{downsample_factor}')

# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (14464, 14464, 20000)

# Voxel offset
offset = (0, 0, 0)

# Layer type
layer_type = 'segmentation'

# segmentation properties in the format of [(number1, label1), (number2, label2) ...]
# where number is an integer that is in the volume and label is a string that describes that segmenetation
sqlController = SqlController('MD589')
db_structures = sqlController.get_structures_dict()

segmentation_properties = [(number, f'{structure}: {label}') for structure, (label, number, sided) in db_structures.items()]
extra_structures = ['MVePC', 'DTgP', 'VTA', 'Li', 'Op', 'RPC', 'MVeMC', 'APT', 'IPR', 'Cb', 
                    'pc', 'SolIM', 'Pr5VL', 'IPC', '8n', 'MPB', 'Pr5', 'DRD', 'VTg', 'IF', 
                    'RR', 'LDTg', '5TT', 'Bar', 'IO', 'Cu', 'SuVe', 'PTg', 'MnR', 'Gr', 
                    'ECu', 'DTgC', 'IPA', 'LPB', 'EW', 'Pr5DM', 'Dk', 'DTg', 'LVe', 'SpVe', 
                    'MVe', 'LSO', 'InC', 'RMC', 'PF', 'CnF', 'Sol', 'IPL', 'X', 'MiTg', 'DRI', 'RPF']
segmentation_properties += [(len(db_structures) + index + 1, structure) for index, structure in enumerate(extra_structures)]

(1366, 1012, 447)


In [4]:
# Run this cell if you want to preview the numpy array before converting to precomputed format
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=resol), 
        voxel_offset=offset
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

http://127.0.0.1:36395/v/aedbc3f54e88332d3fd58c3c48531753d216de2d/


In [10]:
# https://github.com/seung-lab/cloud-volume
# Don't change all compress=False. It seems that Neuroglancer can only read with compress=False
from cloudvolume import CloudVolume

cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = volume.shape[3] if len(volume.shape) > 3 else 1,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = resol, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [64, 64, 64], # units are voxels
    volume_size = volume.shape[:3], # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 2464/2464 [00:11<00:00, 210.48it/s]


In [11]:
import json

vol.info['segment_properties'] = 'names'
vol.commit_info()

segment_properties_path = os.path.join(precompute_path, 'names')
os.makedirs(segment_properties_path, exist_ok=True)

info = {
    "@type": "neuroglancer_segment_properties", 
    "inline": {
        "ids": [str(number) for number, label in segmentation_properties],
        "properties": [{
            "id": "label", 
            "description": "Name of structures",
            "type": "label",
            "values": [str(label) for number, label in segmentation_properties]
        }]
    }
}
with open(os.path.join(segment_properties_path, 'info'), 'w') as file:
    json.dump(info, file, indent=2)

In [ ]:
# https://github.com/seung-lab/igneous
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=True)
tasks = tc.create_downsampling_tasks(cloudpath, compress=False) # Downsample the volumes 
tq.insert(tasks)
tq.execute()

In [ ]:
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False) # The first phase of creating mesh
tq.insert(tasks)
tq.execute()

# It should be able to incoporated to above tasks, but it will give a weird bug. Don't know the reason
tasks = tc.create_mesh_manifest_tasks(cloudpath) # The second phase of creating mesh
tq.insert(tasks)
tq.execute()